## Exploring Ebay Car Sales Data

### Deep Cleaning and Manipulation Using pandas

<img src="https://i.computer-bild.de/imgs/1/1/1/1/0/6/6/5/Ebay-Kleinanzeigen-1024x576-6710c3a77940e514.jpg"/> 
The current project will focus on data scraped from _eBay Kleinanzigen_, a classifieds section of the German eBay website. This dataset is available on [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data) in a slightly modified version (i.e., the current dataset contains a subsample (50,000) of the original submission). 

Aims for this exploration include data cleaning and basic analyses to gather insights regarding the kinds of used cars available for sale and their price points.

Note the accompanying data dictionary for this dataset:

- dateCrawled: When the ad was first crawled.
- name: Name of the car.
- seller: Whether the seller is private or a dealer.
- offerType: Type of listing.
- price: Price on the ad to sell the car.
- abtest: Whether or not the listing is included in an A/B test.
- vehicleType: Type of vehicle.
- year0fRegistration: Year in which the car was first registered.
- gearbox: Transmission type.
- powerPS: Power of the car in PS.
- model: Car model name.
- odometer: Number of kilometers the car has driven.
- month0fRegistration: Month in which car was first registered.
- fuelType: Type of fuel the car uses.
- brand: Brand of the car.
- notRepairedDamage: If the car has damage which has not yet been repaired.
- dateCreated: Date which the eBay listed was created.
- nr0fPictures: Number of pictures in the ad.
- postalCode: Postal code for location of the vehicle.
- lastSeen0nline: When the crawler saw this ad last online.

Let's begin by importing libraries and reading in the dataset:

In [22]:
#Import libraries

import pandas as pd
import numpy as np

#Read in dataset

autos = pd.read_csv("autos.csv", encoding="Latin-1")

In [23]:
#General Info

print(autos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [24]:
print(autos.head())

           dateCrawled                                               name  \
0  2016-03-26 17:47:46                   Peugeot_807_160_NAVTECH_ON_BOARD   
1  2016-04-04 13:38:56         BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik   
2  2016-03-26 18:57:24                         Volkswagen_Golf_1.6_United   
3  2016-03-12 16:58:10  Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...   
4  2016-04-01 14:38:50  Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...   

   seller offerType   price   abtest vehicleType  yearOfRegistration  \
0  privat   Angebot  $5,000  control         bus                2004   
1  privat   Angebot  $8,500  control   limousine                1997   
2  privat   Angebot  $8,990     test   limousine                2009   
3  privat   Angebot  $4,350  control  kleinwagen                2007   
4  privat   Angebot  $1,350     test       kombi                2003   

     gearbox  powerPS   model   odometer  monthOfRegistration fuelType  \
0    manuell      158  andere 

### Preliminary Observations

Briefly, we can see that most of our variables are string (i.e., object) variables. But we can definitely convert more of these to numeric, making some analyses easier. Fortunately, the majority of variables have complete data, but there are some that are missing values, specifically: "vehicleType", "gearbox", "model", "fuelType", and "notRepairedDamage". Also, our variable names (i.e., column headings) are a bit messy, so we will have to rename these variables in a more conventional way.

Looking at the actual data itself, we can see that text information is written in German! It may be prudent to translate some of this information into English."Price" and "odometer" could be converted to numeric by removing any string characters.

#### Renaming Columns 

Let's begin by converting our column names to [snakecase](https://en.wikipedia.org/wiki/Snake_case) (where words are separated by an underscore), from its current presentation, [camelcase](https://en.wikipedia.org/wiki/Camel_case) (where each word in the middle of a phrase begins with a capital letter):  

In [25]:
#Look at column names:

print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [26]:
#Convert column names to usual convention:

new_columns = ['date_crawled', 'name', 'seller', 'offer_type', 
               'price', 'ab_test', 'vehicle_type', 
               'registration_year', 'gearbox', 'power_ps',
               'model', 'odometer', 'registration_month',
               'fuel_type', 'brand', 'unrepaired_damage',
               'ad_created', 'nr_of_pictures', 
               'postal_code', 'last_seen']

#assign updated column names to dataset and check:

autos.columns = new_columns
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


For ease of readability and manipulation, column names should generally be short and presented in lowercase, with underscores separating each word. 

#### Exploration of Variables

Let's continue data exploration by investigating actual data values for each column. Depending on the results, we can remove columns for which all (or almost all) of its values are the same; this suggests low variability and so do not present with useful information:

In [27]:
#Descriptive Statistics

autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-12 16:06:22,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


In [28]:
#Further Investigation of Categorical Variables

print(autos["seller"].value_counts())
print('\n')

print(autos["offer_type"].value_counts())
print('\n')

print(autos["vehicle_type"].value_counts())
print('\n')

print(autos["nr_of_pictures"].value_counts())
print('\n')

print(autos["registration_month"].value_counts())
print('\n')

print(autos["registration_year"].value_counts())

privat        49999
gewerblich        1
Name: seller, dtype: int64


Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64


limousine     12859
kleinwagen    10822
kombi          9127
bus            4093
cabrio         3061
coupe          2537
suv            1986
andere          420
Name: vehicle_type, dtype: int64


0    50000
Name: nr_of_pictures, dtype: int64


0     5075
3     5071
6     4368
5     4107
4     4102
7     3949
10    3651
12    3447
9     3389
11    3360
1     3282
8     3191
2     3008
Name: registration_month, dtype: int64


2000    3354
2005    3015
1999    3000
2004    2737
2003    2727
2006    2708
2001    2703
2002    2533
1998    2453
2007    2304
2008    2231
2009    2098
1997    2028
2011    1634
2010    1597
2017    1453
1996    1444
2012    1323
2016    1316
1995    1313
2013     806
2014     666
1994     660
2018     492
1993     445
2015     399
1990     395
1992     391
1991     356
1989     181
        ... 
1950       3
1955       2
9000    

The _describe_ method gives us the number of unique values for categorical variables, as well as the count for the most frequent class. By investigating categorical variables with a low unique count, we can remove the following variables from further analysis:

- Seller: 49,999 sellers were private.
- offer_type: 49,999 types were offer.
- Number of Pictures: 50,000 ads contained no pictures.

We have also encountered errors along the way that must be further investigated. For example, "registration_year" appears with numerous errors, with a minimal value of 1000 and maximal value of 9999 (with nonsense interim years presented). Also, the minimal value for "registration_month" is 0 (it should be 1, if the maximal value is 12). 

#### Converting Appropriate Columns to Numeric

As previously mentioned, "odometer" and "price" are formatted as string data, but can clearly be converted to numeric data with a bit of tweaking:

In [29]:
#Price: Remove non-numeric characters:

autos["price"] = autos["price"].str.replace("$", "")
autos["price"] = autos["price"].str.replace(",", "")

#Convert Price to numeric:

autos["price"] = autos["price"].astype(int)
print(autos["price"].describe())

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


In [30]:
#Odometer: Remove non-numeric characters:

autos["odometer"] = autos["odometer"].str.replace("km", "")
autos["odometer"] = autos["odometer"].str.replace(",", "")

#Convert Odometer to numeric:

autos["odometer"] = autos["odometer"].astype(int)

#Rename Odometer to Odometer_km

autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
print(autos["odometer_km"].describe())

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


#### Removing Outliers

Next, we will look closer at data that may not look right. As previously mentioned, we can take a look at minimum and maximum values to see if there are data points that don't make sense. If we find anything strange, we can remove these cases.

Let's start by removing the columns that do not contain variable data:

In [31]:
#Drop unnecessary columns:

autos = autos.drop(["seller", "offer_type", "nr_of_pictures"], 
                   axis=1)

In [32]:
#Investigate price:

print(autos["price"].unique().shape)
print('\n')
print(autos["price"].describe())
print('\n')
print(autos["price"].value_counts().sort_index(ascending=True))

(2357,)


count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0           1421
1            156
2              3
3              1
5              2
8              1
9              1
10             7
11             2
12             3
13             2
14             1
15             2
17             3
18             1
20             4
25             5
29             1
30             7
35             1
40             6
45             4
47             1
49             4
50            49
55             2
59             1
60             9
65             5
66             1
            ... 
151990         1
155000         1
163500         1
163991         1
169000         1
169999         1
175000         1
180000         1
190000         1
194000         1
197000         1
198000         1
220000         1
250000         1
259000         1
26

We can see there are some issues with price:

1. We see some very low prices, with 1421 cars selling at 0 Euros. (Although maybe these cars are truly free!)
2. We also see some very high prices, with the most expensive price at 99999999 Euros. 

We have to decide what constitutes an outlier for price.

One statistical method to remove outliers is the [Interquartile Range](https://en.wikipedia.org/wiki/Interquartile_range), which includes the middle 50% of scores, and is equal to the difference between 75th and 25th percentiles of the data. The middle of the IQR is the median, or the middle score of the variable. 

In [33]:
#Find median
print(autos["price"].median())

#Compute interquartile range
Q1 = autos["price"].quantile(0.25)
Q3 = autos["price"].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

2950.0
6100.0


To determine the lower bound of the IQR, divide the IQR by 2 and subtract from the median: (3050 - 2950) = 100

To determine the upper bound of the IQR, divide the IQR by 2 and add to the median: (3050 + 2950) = 6000

Therefore, prices outside of the 100 to 6000 Euro range will be excluded:

In [34]:
#Removing price outliers

autos = autos[autos["price"].between(100,6000)]
print(autos["price"].describe())

count    33547.000000
mean      2239.261782
std       1595.293773
min        100.000000
25%        900.000000
50%       1800.000000
75%       3350.000000
max       6000.000000
Name: price, dtype: float64


We can see that our minimum and maximum are now what we set it to be (100 and 6000), but this reduced our sample size to 33,547 rows.
**Note** that there are numerous ways of determining what an outlier is computationally, each with pros and cons. The IQR is just one possibility.

Let us go ahead and quickly determine whether there are other variables with problematic data, whether incorrectly entered or statistical outliers:

In [35]:
#Investigate registration_year:

print(autos["registration_year"].unique().shape)
print('\n')
print(autos["registration_year"].describe())
print('\n')
print(autos["registration_year"].value_counts().sort_index(ascending=True))

(75,)


count    33547.000000
mean      2002.268191
std         54.102282
min       1000.000000
25%       1998.000000
50%       2001.000000
75%       2005.000000
max       9999.000000
Name: registration_year, dtype: float64


1000       1
1111       1
1800       2
1910       2
1934       2
1937       1
1952       1
1956       2
1958       3
1959       1
1960      11
1961       1
1962       3
1963       2
1964       6
1965       6
1966       7
1967       4
1968       5
1969       7
1970      15
1971       7
1972      11
1973       8
1974       6
1975       4
1976       7
1977       9
1978      20
1979      12
        ... 
1995    1166
1996    1323
1997    1886
1998    2270
1999    2797
2000    2991
2001    2489
2002    2268
2003    2342
2004    2126
2005    2042
2006    1534
2007    1045
2008     744
2009     656
2010     274
2011     132
2012      87
2013      33
2014      25
2015      24
2016    1047
2017    1156
2018     392
2019       2
2800       1
4100       1
4800       1
5000   

In [36]:
#Remove registration_year outliers:

autos = autos[autos["registration_year"].between(1934,2016)]
print(autos["registration_year"].describe())

count    31985.000000
mean      2001.004064
std          5.888581
min       1934.000000
25%       1998.000000
50%       2001.000000
75%       2004.000000
max       2016.000000
Name: registration_year, dtype: float64


In [37]:
#Investigate odometer_km:

print(autos["odometer_km"].unique().shape)
print('\n')
print(autos["odometer_km"].describe())
print('\n')
print(autos["odometer_km"].value_counts().sort_index(ascending=True))

(13,)


count     31985.000000
mean     135836.954823
std       30930.770541
min        5000.000000
25%      150000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


5000        493
10000        38
20000       169
30000       149
40000       151
50000       266
60000       371
70000       440
80000       582
90000       851
100000     1089
125000     3132
150000    24254
Name: odometer_km, dtype: int64


In [38]:
#Investigate registration_month:

print(autos["registration_month"].unique().shape)
print('\n')
print(autos["registration_month"].describe())
print('\n')
print(autos["registration_month"].value_counts().sort_index(ascending=True))

(13,)


count    31985.000000
mean         5.672128
std          3.740741
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64


0     3499
1     2065
2     1905
3     3279
4     2594
5     2648
6     2772
7     2412
8     2007
9     2126
10    2313
11    2121
12    2244
Name: registration_month, dtype: int64


In [39]:
#Remove registration_month outliers:

autos = autos[autos["registration_month"] > 0]
print(autos["registration_month"].describe())

count    28486.000000
mean         6.368848
std          3.357759
min          1.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64


In [40]:
#Investigate fuel_type:

print(autos["fuel_type"].unique().shape)
print('\n')
print(autos["fuel_type"].describe())
print('\n')
print(autos["fuel_type"].value_counts().sort_index(ascending=True))

(8,)


count      26855
unique         7
top       benzin
freq       19664
Name: fuel_type, dtype: object


andere         8
benzin     19664
cng           44
diesel      6660
elektro        8
hybrid         7
lpg          464
Name: fuel_type, dtype: int64


In [41]:
#Investigate unrepaired_damage:

print(autos["unrepaired_damage"].unique().shape)
print('\n')
print(autos["unrepaired_damage"].describe())
print('\n')
print(autos["unrepaired_damage"].value_counts().sort_index(ascending=True))
print('\n')

#Updated dataset characteristics:

autos.describe(include='all')

(3,)


count     23590
unique        2
top        nein
freq      20015
Name: unrepaired_damage, dtype: object


ja       3575
nein    20015
Name: unrepaired_damage, dtype: int64




,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
count,28486,28486,28486.000000,28486,27174,28486.000000,27533,28486.000000,27341,28486.000000,28486.000000,26855,28486,23590,28486,28486.000000,28486
unique,27903,21799,NaN,2,8,NaN,2,NaN,221,NaN,NaN,7,40,2,61,NaN,25271
top,2016-04-02 15:49:30,BMW_316i,NaN,test,kleinwagen,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,2016-04-07 06:17:27
freq,3,75,NaN,14688,8378,NaN,23504,NaN,2314,NaN,NaN,19664,5944,20015,1108,NaN,6
mean,NaN,NaN,2338.779576,NaN,NaN,2001.149301,NaN,100.593695,NaN,136125.465141,6.368848,NaN,NaN,NaN,NaN,50042.428140,NaN
std,NaN,NaN,1600.666117,NaN,NaN,5.648458,NaN,183.214307,NaN,29856.433487,3.357759,NaN,NaN,NaN,NaN,25586.334901,NaN
min,NaN,NaN,100.000000,NaN,NaN,1934.000000,NaN,0.000000,NaN,5000.000000,1.000000,NaN,NaN,NaN,NaN,1067.000000,NaN
25%,NaN,NaN,999.000000,NaN,NaN,1998.000000,NaN,64.000000,NaN,150000.000000,3.000000,NaN,NaN,NaN,NaN,29439.000000,NaN
50%,NaN,NaN,1990.000000,NaN,NaN,2001.000000,NaN,99.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,48599.000000,NaN
75%,NaN,NaN,3500.000000,NaN,NaN,2004.000000,NaN,125.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,69488.000000,NaN


Overall, we removed a few outliers for  "registration_year", and "registration_month". For "registration_year", the minimal value was 1000 and the maximal value was 9999. Clearly these are incorrect, as 1000 was centuries before the invention of the car. Also, because a car can't first be registered after the listing was seen, any vehicle with a registration year above 2016 is inaccurate. For "registration_month", 3499 rows contained a value of '0', which does not make sense as months are generally coded from 1 to 12 and coding from 0 to 12 implies 13 months. Therefore, rows where "registration_month" were coded as zero were omitted.

Our updated number of rows is 28,486. However, there are still some data missing for some variables.

#### Date/Time Exploration:

Let's turn our attention to date/time data. There are five columns that should represent date/time data:

- date_crawled: added by the crawler
- last_seen: added by the crawler
- ad_created: from website
- registration_month: from website
- registration_year: from website

Currently, "date_crawled", "last_seen", and "ad_created" are formatted as string variables, while the other two columns are numeric. We will need to convert these string variables to numeric in order to implement quantitative analyses.

In [42]:
#Lets preview date/time string variables

autos[["date_crawled", "ad_created", "last_seen"]].head(5)

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50
6,2016-03-20 17:55:21,2016-03-20 00:00:00,2016-03-23 02:48:59
7,2016-03-16 18:55:19,2016-03-16 00:00:00,2016-04-07 03:17:32


We can see that, for all three columns, the first ten digits represent the date in 'yyyy-mm-dd' format. There are a few ways that we can pull out these dates and compute counts (or percentages) on them:

In [43]:
#date_crawled: Percentage of cases per date

print(autos["date_crawled"].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True))

2016-03-05    0.024960
2016-03-06    0.014709
2016-03-07    0.037738
2016-03-08    0.034824
2016-03-09    0.034122
2016-03-10    0.032261
2016-03-11    0.034017
2016-03-12    0.036439
2016-03-13    0.014463
2016-03-14    0.036685
2016-03-15    0.034087
2016-03-16    0.029453
2016-03-17    0.032191
2016-03-18    0.012568
2016-03-19    0.033911
2016-03-20    0.037387
2016-03-21    0.037141
2016-03-22    0.033209
2016-03-23    0.032051
2016-03-24    0.029383
2016-03-25    0.030612
2016-03-26    0.031875
2016-03-27    0.030436
2016-03-28    0.034719
2016-03-29    0.034017
2016-03-30    0.033736
2016-03-31    0.032858
2016-04-01    0.032261
2016-04-02    0.035035
2016-04-03    0.038756
2016-04-04    0.036299
2016-04-05    0.013024
2016-04-06    0.003510
2016-04-07    0.001264
Name: date_crawled, dtype: float64


For "date_crawled", we can see that the distribution of percentages per each date are quite similar (from 1-3%), with the exception of April 6th and April 7th, which present with the fewest crawls at < 1%. 

In [44]:
#ad_created: Percentage of cases per date

print(autos["ad_created"].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True))

2015-08-10    0.000035
2015-11-10    0.000035
2015-12-05    0.000035
2015-12-30    0.000035
2016-01-07    0.000035
2016-01-13    0.000035
2016-01-27    0.000035
2016-02-01    0.000035
2016-02-02    0.000070
2016-02-08    0.000035
2016-02-11    0.000035
2016-02-12    0.000070
2016-02-16    0.000035
2016-02-18    0.000070
2016-02-19    0.000035
2016-02-20    0.000070
2016-02-21    0.000070
2016-02-23    0.000035
2016-02-25    0.000035
2016-02-26    0.000035
2016-02-27    0.000211
2016-02-28    0.000140
2016-02-29    0.000105
2016-03-01    0.000140
2016-03-02    0.000035
2016-03-03    0.001018
2016-03-04    0.001615
2016-03-05    0.022608
2016-03-06    0.016008
2016-03-07    0.036474
                ...   
2016-03-09    0.034262
2016-03-10    0.032016
2016-03-11    0.034122
2016-03-12    0.036474
2016-03-13    0.015411
2016-03-14    0.035596
2016-03-15    0.034157
2016-03-16    0.029734
2016-03-17    0.031875
2016-03-18    0.013551
2016-03-19    0.032648
2016-03-20    0.037457
2016-03-21 

We can see that dates in "ad_created" range from August 10, 2015 to April 7, 2016. The percentages of cases per date suggest fewer ads created at the beginning of the dataset (from August 10, 2015 to March 06, 2016), and greater percentages onward (with the exception of April 5, 6 and 7).

In [45]:
#last_seen: Percentage of cases per date

print(autos["last_seen"].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True))

2016-03-05    0.001299
2016-03-06    0.005266
2016-03-07    0.006845
2016-03-08    0.009373
2016-03-09    0.011234
2016-03-10    0.011865
2016-03-11    0.014779
2016-03-12    0.027628
2016-03-13    0.010426
2016-03-14    0.013551
2016-03-15    0.017588
2016-03-16    0.019308
2016-03-17    0.031279
2016-03-18    0.007934
2016-03-19    0.017517
2016-03-20    0.022186
2016-03-21    0.022783
2016-03-22    0.022151
2016-03-23    0.019659
2016-03-24    0.022081
2016-03-25    0.020396
2016-03-26    0.018606
2016-03-27    0.017096
2016-03-28    0.022853
2016-03-29    0.023555
2016-03-30    0.026469
2016-03-31    0.026574
2016-04-01    0.024538
2016-04-02    0.026890
2016-04-03    0.026294
2016-04-04    0.025767
2016-04-05    0.115495
2016-04-06    0.197009
2016-04-07    0.113705
Name: last_seen, dtype: float64


Interestingly, the majority of ads seen occurred during the last three days available in the dataset: April 5-7, with 41% of ads viewed during these three days.

In [46]:
#Descriptives for registration_year:

print(autos["registration_year"].describe())

count    28486.000000
mean      2001.149301
std          5.648458
min       1934.000000
25%       1998.000000
50%       2001.000000
75%       2004.000000
max       2016.000000
Name: registration_year, dtype: float64


Also, after removing outliers for "registration_year", 2001 was shown as the "average" year for all car registrations with a standard deviation of 5 (i.e., ~65% of cases had a car with a registration year between 1996 and 2006.)

#### Exploring Price by Brand:

Let's investigate the "brand" data further:

In [47]:
# Frequencies for each brand:

print(autos["brand"].value_counts())

volkswagen        5944
opel              3791
ford              2365
bmw               2334
mercedes_benz     2281
renault           1777
audi              1742
peugeot           1101
fiat               932
seat               571
smart              514
mazda              496
nissan             478
citroen            467
skoda              419
toyota             406
mitsubishi         290
hyundai            286
volvo              284
honda              249
alfa_romeo         215
kia                202
suzuki             193
sonstige_autos     164
chevrolet          152
chrysler           124
daihatsu            99
mini                97
dacia               66
subaru              66
saab                64
daewoo              63
rover               55
trabant             42
lancia              37
jeep                34
jaguar              29
land_rover          28
lada                21
porsche              8
Name: brand, dtype: int64


The greatest number of car brands for sale is the Volkswagen. We can further aggregate by each brand and compute the average price for each of the brands. Let's focus on  brands that have more than 100 cars available for sale (Volskwagen to Chrysler):

In [48]:
#Separate out our brands of interest:

brands = autos["brand"].value_counts().index[0:25]

#Compute means for each brand:

brand_dict = {}
for b in brands:
    brand_dict[b] = autos.loc[autos["brand"]==b, "price"].mean()

brand_list = pd.Series(brand_dict)
print(brand_list)      

alfa_romeo        2135.651163
audi              2656.589552
bmw               2771.370608
chevrolet         2897.960526
citroen           2395.364026
fiat              1986.344421
ford              1933.556871
honda             2269.469880
hyundai           2801.188811
kia               2736.445545
mazda             2296.935484
mercedes_benz     2837.841736
mitsubishi        2044.037931
nissan            2062.179916
opel              1998.362965
peugeot           2225.656676
renault           1758.529544
seat              2292.688266
skoda             3185.852029
smart             2918.186770
sonstige_autos    2843.231707
suzuki            2592.129534
toyota            3124.899015
volkswagen        2324.717026
volvo             2248.792254
dtype: float64


The brand of car with the highest average selling price is Skoda, followed by Toyota. The least expensive car brand, on average, is Renault.

To simplify things, let's continue our analyses focusing on the top 8 brands, which have more than 1000 listed cars per brand for sale.

#### Exploring Mileage by Brand

We'll focus now on the average mileage for the top 8 brands and see if there's any relationship to their average price:

In [49]:
#Separate out our brands of interest (top 8):

brands = autos["brand"].value_counts().index[0:7]

#Compute price means and mileage for each brand:

brand_dict1 = {}
for b in brands:
    brand_dict1[b] = autos.loc[autos["brand"]==b, "price"].mean()
    
brand_dict2 = {}
for b in brands:
    brand_dict2[b] = autos.loc[autos["brand"]==b, "odometer_km"].mean()

#Convert both dictionaries to series objects

brand_list1 = pd.Series(brand_dict1)
brand_list2 = pd.Series(brand_dict2)

#Create dataframe

brandinfo = pd.DataFrame(brand_list1, columns=['mean_price'])
brandinfo["mean_mileage"] = brand_list2
print(brandinfo)

                mean_price   mean_mileage
audi           2656.589552  146647.531573
bmw            2771.370608  144584.404456
ford           1933.556871  132862.579281
mercedes_benz  2837.841736  143476.545375
opel           1998.362965  136154.049064
renault        1758.529544  133745.075971
volkswagen     2324.717026  140519.010767


Focusing on just these two factors, we would agree that brands with the lowest price _and_ lowest mileage would be our best purchase. Therefore, it appears that the **Renault** brand might be a good choice since it is the least expensive brand (1758 Euros) with the second-lowest mileage (133745km). However, if we pay just a little bit more (1933 Euros), we could purchase a **Ford** car, which exhibits the lowest mileage on average (132862km).

This is just a cursory examination of this dataset! For example, we can also extract information from columns to create new variables, investigate the most common brand/model combinations, or see whether cars with damage are cheaper than their non-damaged counterparts.